# Varational autoencoder by RNN for sudoku sequence generation

## dataset 
- [10,000 solved sudoku](http://www.printable-sudoku-puzzles.com/wfiles/)

## Load sudoku training data

In [152]:
import numpy as np
from glob import glob

import torch
from torch.autograd import Variable
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [153]:

puzzles = sum([open(f).readlines() 
               for f in glob("123456789.txt")], [])
puzzles = [p.strip() for p in puzzles if len(p.strip())==81]
len(puzzles)

10000

In [154]:
puzzles[0]

'123456789578139624496872153952381467641297835387564291719623548864915372235748916'

### check they are valid sudoku

In [155]:
def check_sudoku(puzzle):
    assert len(puzzle) == 81
    p = np.array(list(puzzle)).reshape(9, 9)
    rows = range(9)
    cols = range(9)
    digits = set('123456789')
    strides = [slice(0, 3), slice(3, 6), slice(6, 9)]
    squares = [(r, c) for r in strides for c in strides]
    for r in rows:
        assert set(p[r,:]) == digits, "err: row %i" % r
    for c in cols:
        assert set(p[:,c]) == digits, "err: col %i" % c
    for sr, sc in squares:
        assert set(p[sr, sc].ravel()) == digits, "err: sqr %i %i" % (sr, sc)

In [156]:
# negative example
p = ''.join(['123456789'] * 9)
print(p)
check_sudoku(p)

123456789123456789123456789123456789123456789123456789123456789123456789123456789


AssertionError: err: col 0

In [157]:
for p in puzzles:
    check_sudoku(p)

## Data Processing

### data augumentation

In [158]:
from tqdm import tqdm_notebook

In [159]:
%%time
def augument(puzzles):
    augumented = [p for p in puzzles]
#     augumented += [p[::-1] for p in puzzles]
    for p in puzzles:
        p = np.array(list(p)).reshape([9, 9])
        augumented.append(''.join(np.fliplr(p).ravel()))
        p = p.T
        augumented.append(''.join(p.ravel()))
        augumented.append(''.join(np.fliplr(p).ravel()))
        p = p.T
        augumented.append(''.join(p.ravel()))
        augumented.append(''.join(np.fliplr(p).ravel()))
        p = p.T
        augumented.append(''.join(p.ravel()))
        augumented.append(''.join(np.fliplr(p).ravel()))
    return augumented

puzzles = augument(puzzles)

CPU times: user 1.41 s, sys: 104 ms, total: 1.52 s
Wall time: 1.44 s


In [160]:
print(len(puzzles))

for p in puzzles:
    check_sudoku(p)

80000


In [161]:
symbol2index = dict(zip('123456789', range(9)))
index2symbol = dict(zip(range(9), '123456789'))

def puzzle2tensor(puzzle_batch):
    batch_size = len(puzzle_batch)
    seq_len = len(puzzle_batch[0])
    t = torch.zeros([batch_size, seq_len, 9])
    for r in range(batch_size):
        for c in range(seq_len):
            s = symbol2index[puzzle_batch[r][c]]
            t[r, c, s] = 1
    return t

def puzzle2target(puzzle_batch):
    batch_size = len(puzzle_batch)
    seq_len = len(puzzle_batch[0])
    t = torch.LongTensor(batch_size, seq_len).zero_()
    for r in range(batch_size):
        for c in range(seq_len):
            s = symbol2index[puzzle_batch[r][c]]
            t[r, c] = s
    return t

def tensor2puzzle(tensors):
    """tensors.size() == [batch_size, seq, 9]
    """
    _, p = tensors.max(dim=2)
    p = p.squeeze().numpy()
    puzzles = []
    for r in p:
        puzzles.append(''.join([index2symbol.get(s) for s in r]))
    return puzzles

def output2puzzle(y):
    """y.size() == [batch_size, 9]
    """
    _, labels = y.max(dim=1)
    return labels.numpy().squeeze() + 1

In [162]:
## test
t = puzzle2tensor([p[:5] for p in puzzles[:2]])
p = tensor2puzzle(t)
print(t.size())
print(p)

torch.Size([2, 5, 9])
['12345', '12345']


In [163]:
## test
puzzle2target([p[:5] for p in puzzles[:2]])


 0  1  2  3  4
 0  1  2  3  4
[torch.LongTensor of size 2x5]

## Models - seq prediction 

In [164]:
class RnnVA(nn.Module):
    """RNN based Variational Autoencoder
    """
    def __init__(self):
        super(RnnVA, self).__init__()
        self.input_size = 9
        self.seq_len = 81
        self.encoder_rnn_num_layers = 2
        self.encoder_rnn_hidden_size = 128
        self.encoder_fc1_hidden_size = 256
        self.va_size = 256
        self.decoder_fc1_hidden_size = 256
        self.decoder_rnn_num_layers = 1
        self.decoder_rnn_hidden_size = 128
        self.output_size = 9
        
        self.encoder_rnn = nn.GRU(input_size=self.input_size,
                                  hidden_size=self.encoder_rnn_hidden_size,
                                  num_layers=self.encoder_rnn_num_layers,
                                  batch_first=True,
                                  bidirectional=False)
        self.encoder_fc1 = nn.Linear(self.encoder_rnn_hidden_size,
                                    self.encoder_fc1_hidden_size)
        self.elu = nn.ELU()
        self.va_mean = nn.Linear(self.encoder_fc1_hidden_size, self.va_size)
        self.va_gamma = nn.Linear(self.encoder_fc1_hidden_size, self.va_size)
        self.decoder_fc1 = nn.Linear(self.va_size, self.decoder_fc1_hidden_size)
        self.decoder_rnn = nn.GRU(input_size=self.decoder_fc1_hidden_size,
                                 hidden_size=self.decoder_rnn_hidden_size,
                                 num_layers=self.decoder_rnn_num_layers,
                                 batch_first=True,
                                 bidirectional=False)
        self.decoder_classifier = nn.Linear(self.decoder_rnn_hidden_size,
                                            self.output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        batch_size = x.size(0)
        seq_len = x.size(1)
        encode_h0 = Variable(torch.zeros([self.encoder_rnn_num_layers,
                                   batch_size,
                                   self.encoder_rnn_hidden_size])).cuda()
        out, h = self.encoder_rnn(x, encode_h0)
        out = out.contiguous().view([-1, self.encoder_rnn_hidden_size])
        out = self.elu(self.encoder_fc1(out))
        self.mean = self.va_mean(out)
        self.gamma = self.va_gamma(out)
        self.sigma = torch.exp(self.gamma * .5)
        noise = Variable(torch.randn(self.sigma.size())).cuda()
        h = self.mean + self.sigma * noise
        out = self.elu(self.decoder_fc1(h))
        out = out.view([batch_size, seq_len, -1])
        decode_h0 = Variable(torch.zeros([self.decoder_rnn_num_layers,
                                   batch_size,
                                   self.decoder_rnn_hidden_size])).cuda()
        out, h = self.decoder_rnn(out, decode_h0)
        out = out.contiguous().view([-1, self.decoder_rnn_hidden_size])
        logits = self.decoder_classifier(out)
        probs = self.sigmoid(logits)
        probs = probs.view([batch_size, seq_len, -1])
        return probs
    def generate(self, batch_size): 
        seq_len = 81
        h = Variable(torch.randn([batch_size * seq_len, self.va_size])).cuda()
        out = self.elu(self.decoder_fc1(h))
        out = out.view([batch_size, seq_len, -1])
        decode_h0 = Variable(torch.zeros([self.decoder_rnn_num_layers,
                                   batch_size,
                                   self.decoder_rnn_hidden_size])).cuda()
        out, h = self.decoder_rnn(out, decode_h0)
        out = out.contiguous().view([-1, self.decoder_rnn_hidden_size])
        logits = self.decoder_classifier(out)
        probs = self.sigmoid(logits)
        probs = probs.view([batch_size, seq_len, -1])
        return probs

In [165]:
model = RnnVA().cuda()
x = Variable(torch.zeros([32, 81, 9])).cuda()
y = model(x)
y.size(), model.mean.size(), model.gamma.size()

(torch.Size([32, 81, 9]), torch.Size([2592, 256]), torch.Size([2592, 256]))

In [166]:
model.eval()
generated = model.generate(10)
print(generated.size())
tensor2puzzle(generated.cpu().data)

torch.Size([10, 81, 9])


['777777772224444777777777777444666666667777744667777744774444444772777774447466477',
 '777227744644877777777444466666642276664363337777776644444777647733377722246447777',
 '777777766666623427747727777777444444474444774477777744444333366666644477771444466',
 '477446644422777777747444444777777777766677777777444444777747777477777744447767766',
 '766667772222666622272268782444474774444444677774444777766667444447777744448844144',
 '477772277444444444444446644668464677783747777444777777473776667777777777777776444',
 '447274444444447777777774444444444777276666277766667447444466742922244447777777666',
 '777444444444444444477444444488984444444444464446666624767777777777726666677666664',
 '777777774277762477444477777777777477444444447444444477777444444444432284444744444',
 '777774444444444444444426644477744444444437776444777777477777777777744444444444444']

## Data Preparation

In [167]:
class SudokuDataSet(Dataset):
    def __init__(self, puzzles):
        self.puzzles = puzzle2tensor(puzzles)
    def __len__(self):
        return self.puzzles.size(0)
    def __getitem__(self, i):
        return self.puzzles[i, ...]

In [168]:
data = SudokuDataSet(puzzles)


In [169]:
model = RnnVA().cuda()
model.train()

RnnVA (
  (encoder_rnn): GRU(9, 128, num_layers=2, batch_first=True)
  (encoder_fc1): Linear (128 -> 256)
  (elu): ELU (alpha=1.0)
  (va_mean): Linear (256 -> 256)
  (va_gamma): Linear (256 -> 256)
  (decoder_fc1): Linear (256 -> 256)
  (decoder_rnn): GRU(256, 128, num_layers=2, batch_first=True)
  (decoder_classifier): Linear (128 -> 9)
  (sigmoid): Sigmoid ()
)

In [170]:
n_epochs = 10

batches = DataLoader(data, batch_size=128, shuffle=True, num_workers=4)
xentropy = nn.BCELoss()


optimizer = optim.Adam(model.parameters())

for epoch in range(n_epochs):
    for b, batch in enumerate(batches):
        x = Variable(batch).cuda()
        
        model.zero_grad()
        xx = model(x)
        restore_loss = xentropy(xx.view([-1, 9]), x.view([-1, 9]))
        latent_loss = 0.5 * torch.mean(torch.exp(model.gamma) + model.mean*model.mean -1 - model.gamma)
        loss = restore_loss + latent_loss
        loss.backward()
        optimizer.step()
        
        if b % 100 == 0:
            print(epoch, b, loss.data[0], restore_loss.data[0], latent_loss.data[0])

0 0 0.6830543875694275 0.6810584664344788 0.0019959351047873497
0 100 0.34842395782470703 0.3481610417366028 0.00026292531401850283
0 200 0.2610192596912384 0.24240830540657043 0.01861095055937767
0 300 0.18023937940597534 0.15498310327529907 0.02525627426803112
0 400 0.10254645347595215 0.09067604690790176 0.011870408430695534
0 500 0.0719316154718399 0.05945393443107605 0.012477677315473557
0 600 0.04055941477417946 0.02844056487083435 0.012118849903345108
1 0 0.034348711371421814 0.022412899881601334 0.011935810558497906
1 100 0.02357545495033264 0.011876785196363926 0.011698668822646141
1 200 0.020067691802978516 0.008237470872700214 0.011830220930278301
1 300 0.01766396313905716 0.0060273571871221066 0.01163660641759634
1 400 0.01676705665886402 0.005118110217154026 0.011648946441709995
1 500 0.015445093624293804 0.0035015754401683807 0.011943518184125423
1 600 0.015106558799743652 0.003324800403788686 0.01178175862878561
2 0 0.01575370691716671 0.0037397209089249372 0.01201398670

In [174]:
model.eval()
generated_puzzles = tensor2puzzle(model.generate(10).cpu().data)

In [175]:
generated_puzzles

['977856965613696834762948938529125533971929512375428211378312374639111888222825415',
 '921131418648989574287467361651932779741598398738375813632876422128283715694314173',
 '678162561398716893736797614695129491812871893343675277876328838262615394465697797',
 '725367329277369568121959158662539611951461578567515366345555653641991557129113856',
 '754615739151459197688665288213645681389671845961267562713977495257214837126528899',
 '614986244291572669535299616186842924995529585882287252185496531717297777649593189',
 '132942745985871726792861756781725638945627488311646171754798518249762224545726215',
 '662322576655858124472946965821193531746769297179354439191726349177964866756666318',
 '625451418689394171637487922753631923571595939482241176721471355883521567855851865',
 '125493243367948598294794697727849619326333585988929778112721725347973617415142499']

In [177]:
check_sudoku(generated_puzzles[-3])

AssertionError: err: row 0